#### Higher school of econimics
#### Name: Zhornichenko Ilya
#### Group: TMSS 


In [ ]:
# when running in colab, un-comment this
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall19/week03_convnets/cifar.py

In [2]:
import numpy as np
from cifar import load_cifar10
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10("cifar_data")

class_names = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer',
                        'dog', 'frog', 'horse', 'ship', 'truck'])

print(X_train.shape,y_train.shape)

Dataset not found. Downloading...
(40000, 3, 32, 32) (40000,)


In [3]:
from glob import glob
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T, models
from torch import optim
from torchvision import models
import torch
from torch.optim import lr_scheduler
import time
import copy
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import normalized_mutual_info_score
import seaborn as sns
from torchvision import transforms, models

In [4]:
n_epochs = 100
batch_size_train = 32
batch_size_test = 32
lr = 0.001
momentum = 0.5
log_interval = 10

In [5]:
class DataSet(Dataset):
  def __init__(self, images, labels):
    self.images = images
    self.labels = labels
    self.my_transforms = transforms.Compose([
    transforms.ToTensor()
    ])


  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img = self.my_transforms(np.transpose(self.images[idx], (1, 2, 0)))
    return img, self.labels[idx]

In [6]:
train_loader = torch.utils.data.DataLoader(DataSet(X_train, y_train), batch_size=batch_size_train , shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(DataSet(X_test, y_test), batch_size=batch_size_test , shuffle=True, pin_memory=True)

In [7]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [8]:
example_data.shape

torch.Size([32, 3, 32, 32])

In [9]:
dataloaders = {
    'train':train_loader, 
    'val': test_loader
}
dataset_sizes = {
    'train':X_train.shape[0],
    'val':X_test.shape[0]
}

In [10]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [11]:
net = Net()

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu = torch.device('cpu')

In [13]:
net = net.to(device)

In [14]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [15]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device).long()
                # print(inputs.shape)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)


            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
train_model(net, criterion, optimizer,  num_epochs=n_epochs)

Epoch 0/99
----------
train Loss: 2.3024 Acc: 0.1101
val Loss: 2.3009 Acc: 0.1573

Epoch 1/99
----------
train Loss: 2.2932 Acc: 0.1521
val Loss: 2.2603 Acc: 0.1329

Epoch 2/99
----------
train Loss: 2.1171 Acc: 0.2052
val Loss: 1.9971 Acc: 0.2583

Epoch 3/99
----------
train Loss: 1.9558 Acc: 0.2792
val Loss: 1.8473 Acc: 0.3221

Epoch 4/99
----------
train Loss: 1.7807 Acc: 0.3543
val Loss: 1.7003 Acc: 0.3784

Epoch 5/99
----------
train Loss: 1.6487 Acc: 0.4010
val Loss: 1.5634 Acc: 0.4311

Epoch 6/99
----------
train Loss: 1.5516 Acc: 0.4400
val Loss: 1.5158 Acc: 0.4461

Epoch 7/99
----------
train Loss: 1.4917 Acc: 0.4615
val Loss: 1.4540 Acc: 0.4655

Epoch 8/99
----------
train Loss: 1.4386 Acc: 0.4825
val Loss: 1.4169 Acc: 0.4839

Epoch 9/99
----------
train Loss: 1.3960 Acc: 0.4949
val Loss: 1.3940 Acc: 0.4929

Epoch 10/99
----------
train Loss: 1.3604 Acc: 0.5124
val Loss: 1.3605 Acc: 0.5054

Epoch 11/99
----------
train Loss: 1.3310 Acc: 0.5234
val Loss: 1.3320 Acc: 0.5172

Ep

KeyboardInterrupt: ignored

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(128, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net2 = Net2()
net2 = net2.to(device)
criterion2 = nn.CrossEntropyLoss()
optimizer2 = optim.SGD(net2.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_model(net2, criterion2, optimizer2,  num_epochs=n_epochs)

Epoch 0/99
----------
train Loss: 2.3030 Acc: 0.1019
val Loss: 2.3023 Acc: 0.1000

Epoch 1/99
----------
train Loss: 2.3022 Acc: 0.1058
val Loss: 2.3019 Acc: 0.1268

Epoch 2/99
----------
train Loss: 2.3015 Acc: 0.1212
val Loss: 2.3007 Acc: 0.1500

Epoch 3/99
----------
train Loss: 2.2990 Acc: 0.1322
val Loss: 2.2949 Acc: 0.1402

Epoch 4/99
----------
train Loss: 2.2638 Acc: 0.1515
val Loss: 2.1710 Acc: 0.1758

Epoch 5/99
----------
train Loss: 2.0987 Acc: 0.1977
val Loss: 2.0438 Acc: 0.2267

Epoch 6/99
----------
train Loss: 2.0157 Acc: 0.2279
val Loss: 1.9725 Acc: 0.2334

Epoch 7/99
----------
train Loss: 1.9398 Acc: 0.2527
val Loss: 1.8837 Acc: 0.2626

Epoch 8/99
----------
train Loss: 1.8504 Acc: 0.2953
val Loss: 1.7986 Acc: 0.3173

Epoch 9/99
----------
train Loss: 1.7588 Acc: 0.3468
val Loss: 1.6922 Acc: 0.3753

Epoch 10/99
----------
train Loss: 1.6785 Acc: 0.3860
val Loss: 1.6324 Acc: 0.4003

Epoch 11/99
----------
train Loss: 1.6215 Acc: 0.4072
val Loss: 1.5693 Acc: 0.4260

Ep

Net2(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
class Net3(nn.Module):
  def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.batchNorm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.batchNorm2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
      x = self.pool(F.relu(self.batchNorm1(self.conv1(x))))
      x = self.pool(F.relu(self.batchNorm2(self.conv2(x))))
      x = torch.flatten(x, 1) # flatten all dimensions except batch
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

In [ ]:
net3 = Net3()
net3 = net3.to(device)
criterion3 = nn.CrossEntropyLoss()
optimizer3 = optim.SGD(net3.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_model(net3, criterion3, optimizer3,  num_epochs=n_epochs)

Epoch 0/99
----------
train Loss: 1.8349 Acc: 0.3371
val Loss: 1.5999 Acc: 0.3897

Epoch 1/99
----------
train Loss: 1.4371 Acc: 0.4775
val Loss: 1.5773 Acc: 0.4207

Epoch 2/99
----------
train Loss: 1.2916 Acc: 0.5369
val Loss: 1.3068 Acc: 0.5357

Epoch 3/99
----------
train Loss: 1.1964 Acc: 0.5756
val Loss: 1.1560 Acc: 0.5911

Epoch 4/99
----------
train Loss: 1.1258 Acc: 0.6016
val Loss: 1.1732 Acc: 0.5815

Epoch 5/99
----------
train Loss: 1.0732 Acc: 0.6235
val Loss: 1.0736 Acc: 0.6244

Epoch 6/99
----------
train Loss: 1.0262 Acc: 0.6375
val Loss: 1.0768 Acc: 0.6192

Epoch 7/99
----------
train Loss: 0.9854 Acc: 0.6535
val Loss: 1.0597 Acc: 0.6268

Epoch 8/99
----------
train Loss: 0.9458 Acc: 0.6698
val Loss: 1.0279 Acc: 0.6326

Epoch 9/99
----------
train Loss: 0.9148 Acc: 0.6777
val Loss: 1.0382 Acc: 0.6368

Epoch 10/99
----------
train Loss: 0.8842 Acc: 0.6913
val Loss: 0.9533 Acc: 0.6658

Epoch 11/99
----------
train Loss: 0.8527 Acc: 0.7024
val Loss: 0.9753 Acc: 0.6583

Ep

KeyboardInterrupt: ignored

In [ ]:
net3 = Net3()
net3 = net3.to(device)
criterion3 = nn.CrossEntropyLoss()
optimizer3 = optim.Adam(net3.parameters(), lr=0.001)

In [ ]:
train_model(net3, criterion3, optimizer3,  num_epochs=n_epochs)

Epoch 0/99
----------
train Loss: 1.5068 Acc: 0.4494
val Loss: 1.3117 Acc: 0.5207

Epoch 1/99
----------
train Loss: 1.2462 Acc: 0.5511
val Loss: 1.2599 Acc: 0.5448

Epoch 2/99
----------
train Loss: 1.1287 Acc: 0.5970
val Loss: 1.2411 Acc: 0.5597

Epoch 3/99
----------
train Loss: 1.0450 Acc: 0.6310
val Loss: 1.1093 Acc: 0.6096

Epoch 4/99
----------
train Loss: 0.9828 Acc: 0.6523
val Loss: 1.2477 Acc: 0.5624

Epoch 5/99
----------
train Loss: 0.9267 Acc: 0.6718
val Loss: 1.0273 Acc: 0.6455

Epoch 6/99
----------
train Loss: 0.8814 Acc: 0.6874
val Loss: 1.0595 Acc: 0.6309

Epoch 7/99
----------
train Loss: 0.8437 Acc: 0.7046
val Loss: 1.1252 Acc: 0.6118

Epoch 8/99
----------
train Loss: 0.8083 Acc: 0.7144
val Loss: 1.0643 Acc: 0.6314

Epoch 9/99
----------
train Loss: 0.7746 Acc: 0.7263
val Loss: 1.1169 Acc: 0.6270

Epoch 10/99
----------
train Loss: 0.7460 Acc: 0.7371
val Loss: 1.0152 Acc: 0.6530

Epoch 11/99
----------
train Loss: 0.7135 Acc: 0.7484
val Loss: 1.0416 Acc: 0.6539

Ep

KeyboardInterrupt: ignored

In [ ]:
class Net4(nn.Module):
  def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.batchNorm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.batchNorm2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
      x = self.pool(F.relu(self.batchNorm1(self.conv1(x))))
      x = self.pool(F.relu(self.batchNorm2(self.conv2(x))))
      x = torch.flatten(x, 1) # flatten all dimensions except batch
      x = F.sigmoid(self.fc1(x))
      x = F.sigmoid(self.fc2(x))
      x = self.fc3(x)
      return x

In [ ]:
net4 = Net4()
net4 = net4.to(device)
criterion4 = nn.CrossEntropyLoss()
optimizer4 = optim.SGD(net4.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_model(net4, criterion4, optimizer4,  num_epochs=n_epochs)

Epoch 0/99
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train Loss: 2.2961 Acc: 0.1266
val Loss: 2.2753 Acc: 0.1840

Epoch 1/99
----------
train Loss: 2.1950 Acc: 0.1851
val Loss: 2.0926 Acc: 0.1850

Epoch 2/99
----------
train Loss: 1.9673 Acc: 0.2041
val Loss: 1.9159 Acc: 0.2104

Epoch 3/99
----------
train Loss: 1.9019 Acc: 0.2184
val Loss: 1.9665 Acc: 0.2160

Epoch 4/99
----------
train Loss: 1.8767 Acc: 0.2434
val Loss: 1.9700 Acc: 0.2194

Epoch 5/99
----------
train Loss: 1.8472 Acc: 0.2750
val Loss: 1.8949 Acc: 0.2540

Epoch 6/99
----------
train Loss: 1.7912 Acc: 0.3046
val Loss: 1.7499 Acc: 0.3053

Epoch 7/99
----------
train Loss: 1.7206 Acc: 0.3322
val Loss: 1.7232 Acc: 0.3107

Epoch 8/99
----------
train Loss: 1.6667 Acc: 0.3569
val Loss: 1.8512 Acc: 0.2951

Epoch 9/99
----------
train Loss: 1.6198 Acc: 0.3805
val Loss: 1.7887 Acc: 0.2950

Epoch 10/99
----------
train Loss: 1.5782 Acc: 0.4048
val Loss: 1.5496 Acc: 0.4287

Epoch 11/99
----------
train Loss: 1.5281 Acc: 0.4290
val Loss: 1.4828 Acc: 0.4483

Epoch 12/99
----------
t

Net4(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batchNorm1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [16]:
resnet = models.resnet18(pretrained=False)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)
resnet.to(device)
criterion_res = nn.CrossEntropyLoss()
optimizer_res = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [17]:
train_model(resnet, criterion_res, optimizer_res,  num_epochs=n_epochs)

Epoch 0/99
----------
train Loss: 1.6228 Acc: 0.4115
val Loss: 1.4019 Acc: 0.4893

Epoch 1/99
----------
train Loss: 1.2891 Acc: 0.5380
val Loss: 1.1808 Acc: 0.5728

Epoch 2/99
----------
train Loss: 1.0991 Acc: 0.6071
val Loss: 1.1357 Acc: 0.5982

Epoch 3/99
----------
train Loss: 0.9539 Acc: 0.6640
val Loss: 1.1329 Acc: 0.6058

Epoch 4/99
----------
train Loss: 0.8324 Acc: 0.7056
val Loss: 1.0408 Acc: 0.6425

Epoch 5/99
----------
train Loss: 0.7159 Acc: 0.7472
val Loss: 1.1156 Acc: 0.6241

Epoch 6/99
----------
train Loss: 0.6186 Acc: 0.7799
val Loss: 1.1240 Acc: 0.6359

Epoch 7/99
----------
train Loss: 0.5278 Acc: 0.8131
val Loss: 1.1306 Acc: 0.6428

Epoch 8/99
----------
train Loss: 0.4575 Acc: 0.8356
val Loss: 1.1255 Acc: 0.6596

Epoch 9/99
----------
train Loss: 0.3754 Acc: 0.8657
val Loss: 1.2724 Acc: 0.6505

Epoch 10/99
----------
train Loss: 0.3154 Acc: 0.8879
val Loss: 1.3206 Acc: 0.6433

Epoch 11/99
----------
train Loss: 0.2729 Acc: 0.9022
val Loss: 1.3080 Acc: 0.6572

Ep

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  